In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim

import numpy as np
import os
import shutil
import argparse
import pdb

import sys
sys.path.insert(0, '../SSL-for-HSI-classification/')
from utils import metrics, sample_gt, softmax,calprecision
from datasets import get_dataset, get_originate_dataset
from DenseConv import *
import argparse
from skimage.segmentation import felzenszwalb
import time
from scipy.stats import entropy
import matplotlib.pyplot as plt

In [38]:
def sample_gt3(img, gt, train_gt, test_gt, SAMPLE_PERCENTAGE, IGNORED_LABELS):
    X = img
    Y = gt
    labels = np.unique(Y)
    labels = np.array([val for val in labels if not val in IGNORED_LABELS])
    print(labels)
    row, col, n_band = X.shape
    # num_class = np.max(Y) # TODO: Fix to the correct amount
    num_class = len(np.unique(labels))
    for i, val in enumerate(labels): # range(1, num_class + 1):
        index = np.where(train_gt == val)
        index2 = np.where(test_gt == val)

        if i == 0:
            array1_train = index[0]
            array2_train = index[1]
            array1_test = index2[0]
            array2_test = index2[1]
        else:
            array1_train = np.concatenate((array1_train, index[0]))
            array2_train = np.concatenate((array2_train, index[1]))
            array1_test = np.concatenate((array1_test, index2[0]))
            array2_test = np.concatenate((array2_test, index2[1]))

    y_train = Y[array1_train, array2_train]
    trueEDtimesSID = []
    trueEDtimesSID2 = []
    sumtrueES = []
    sumfalseES = []
    pseudo_labels3 = np.zeros([row, col, num_class + 1])
    for i in range(0, len(array1_test)):
        if i % 1000 == 0:
            print("i:%d" % (i))
        #if i%200!=0:
        #    continue
        xtest = array1_test[i]
        ytest = array2_test[i]
        labeltest = Y[xtest, ytest]
        specvectortest = X[xtest, ytest]
        # i
        EDs = np.zeros(num_class)
        SIDs = np.zeros(num_class)
        EDtimesSIDs = np.zeros(num_class)
        EDtimesSIDs2 = np.zeros(num_class)
        minED = 10000000000
        for j, val in enumerate(labels): #range(1, num_class + 1):  # 类别循环
            # print(j)
        #for j in np.unique(gt) + 1:
            index2 = np.where(y_train == val)  ## 当前类别序号
            index2 = index2[0]
            EDsclass = []
            SIDclass = []
            EDtimesSIDclass = []
            for nn in range(0, len(index2)):  # 类别内训练集循环 nn
                # print(index2[nn])##当前训练样本序号
                ind = index2[nn]
                xtrain = array1_train[ind]
                ytrain = array2_train[ind]
                specvectortrain = X[xtrain, ytrain]
                ED = np.sqrt(np.square(xtest - xtrain) + np.square(ytest - ytrain))
                SID1 = entropy(specvectortest, specvectortrain)
                SID2 = entropy(specvectortrain, specvectortest)
                SID = SID1 + SID2
                EDtimesSID = np.sqrt(ED * SID)
                ED = ED + SID
                EDsclass.append(ED)
                SIDclass.append(SID)
                EDtimesSIDclass.append(EDtimesSID)

                if ED < minED:
                    minED = ED
            # =================================
            inde = np.argsort(EDsclass)

            jiaquan = 0
            for nn in range(0, len(index2)):
                jiaquandis = EDsclass[inde[nn]] * (float(num_class) ** (-nn))  # 类别内训练集循环 nn
                jiaquan = jiaquan + jiaquandis

            #EDs[j - 1] = jiaquan
            EDs[j] = jiaquan
            #SIDs[j - 1] = np.min(SIDclass)
            SIDs[j] = np.min(SIDclass)
            #EDtimesSIDs[j - 1] = np.min(EDtimesSIDclass)
            EDtimesSIDs[j] = np.min(EDtimesSIDclass)
            ###
            jiaquan2 = 0
            inde2 = np.argsort(EDtimesSIDclass)
            for nn in range(0, len(index2)):
                jiaquandis = EDtimesSIDclass[inde2[nn]] * (float(num_class) ** (-nn))  # 类别内训练集循环 nn
                jiaquan2 = jiaquan2 + jiaquandis
            # EDtimesSIDs2[j - 1] = jiaquan2
            EDtimesSIDs2[j] = jiaquan2
        ###
        # ========================
        # print("minED:", minED)
        # if minED>2.71:
        if np.min(EDtimesSIDs) > 0.085:
            continue
        else:
            minn = np.min(EDs)
            softm = softmax(16 / EDs)
            softm2 = softmax(-EDs * num_class)
            softm3 = softmax(-EDtimesSIDs2 * num_class*100)
            minn

        labeEDtimesSIDs = np.argmin(EDtimesSIDs) + 1
        labeEDtimesSIDs2 = np.argmin(EDtimesSIDs2) + 1
        train_gt[xtest, ytest] = labeEDtimesSIDs2
        pseudo_labels3[xtest, ytest][1:len(softm3) + 1] = softm3


        if labeEDtimesSIDs == labeltest:
            trueEDtimesSID.append(1)
            sumtrueES.append(np.min(EDtimesSIDs))
        else:
            trueEDtimesSID.append(0)
            sumfalseES.append(np.min(EDtimesSIDs))
            print("falseEDtimesSID:", np.min(EDtimesSIDs))

        if labeEDtimesSIDs2 == labeltest:
            trueEDtimesSID2.append(1)
        else:
            trueEDtimesSID2.append(0)

    accuEDtimesSID2 = np.sum(trueEDtimesSID2) / len(trueEDtimesSID2)

    print("lenEDtimesSID2: %d,accurate:%f, truenum:%d" % (
    len(trueEDtimesSID2), 100 * accuEDtimesSID2, np.sum(trueEDtimesSID2)))
    return train_gt,pseudo_labels3



In [ ]:
datasets_root = '/mnt/data/leevi/'
DATASET = 'IndianPines'
SAMPLE_PERCENTAGE = 5
img, gt, LABEL_VALUES, IGNORED_LABELS, _, _ = get_dataset(DATASET, datasets_root)
IGNORED_LABELS = []
X, Y = get_originate_dataset(DATASET, datasets_root)
train_gt, test_gt = sample_gt(gt, SAMPLE_PERCENTAGE, mode='fixed')
_, pseudo_labels3 = sample_gt3(X, Y, train_gt, test_gt, SAMPLE_PERCENTAGE, IGNORED_LABELS)


/home/leevi/notebooks/../SSL-for-HSI-classification/utils.py:477: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  train_gt[train_indices] = gt[train_indices]
/home/leevi/notebooks/../SSL-for-HSI-classification/utils.py:478: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_gt[test_indices] = gt[test_indices]


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
i:0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0
falseEDtimesSID: 0.0


In [ ]:
fig = plt.figure(figsize=(5,5))
plt.imshow(gt, aspect=0.1)
plt.colorbar()
fig.show()

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.imshow(img[:,:,100], aspect=0.1)
plt.colorbar()
fig.show()

In [ ]:
for i in np.unique(gt):
    print(LABEL_VALUES[i])

In [ ]:
IGNORED_LABELS

In [11]:
LABEL_VALUES_2 = []
for i, val in enumerate(np.unique(gt)):
    if val not in IGNORED_LABELS:
        x, y = np.where(gt==val)
        for j in zip(x,y):
            gt[j[0], j[1]] = i
        x, y = np.where(Y==val)
        for j in zip(x,y):
            Y[j[0], j[1]] = i
        LABEL_VALUES_2.append(LABEL_VALUES[val])
LABEL_VALUES_2

['High intensity developed',
 'Med-low intensity developed',
 'Deciduous, Evergreen, mixed forest',
 'shrubland',
 'Grassland-Pasture',
 'Bareland',
 'water',
 'corn',
 'cereals',
 'almonds',
 'grass fodders',
 'vinewards-grapes',
 'walnuts',
 'pistachios',
 'citrus',
 'fallow']

In [12]:
train_gt, test_gt = sample_gt(gt, SAMPLE_PERCENTAGE, mode='fixed')
_, pseudo_labels3 = sample_gt3(X, Y, train_gt, test_gt, SAMPLE_PERCENTAGE)


i:0
i:1000
i:2000
i:3000
i:4000
i:5000
i:6000
i:7000
i:8000
i:9000
i:10000
i:11000
i:12000
i:13000
i:14000
i:15000
i:16000


/home/leevi/notebooks/../SSL-for-HSI-classification/utils.py:505: RuntimeWarning: invalid value encountered in subtract
  x = x - x_row_max


i:17000
i:18000
i:19000
i:20000
i:21000
i:22000
i:23000
i:24000
i:25000
i:26000
i:27000
i:28000
i:29000
i:30000
i:31000
i:32000
i:33000
i:34000
i:35000
i:36000
i:37000
i:38000
i:39000
i:40000
i:41000
i:42000
i:43000
i:44000
i:45000
i:46000
i:47000
i:48000
i:49000
i:50000
i:51000
i:52000
i:53000
i:54000
i:55000
i:56000
i:57000
i:58000
i:59000
i:60000
i:61000
i:62000
i:63000
i:64000
i:65000
i:66000
i:67000
i:68000
i:69000
i:70000
i:71000
i:72000
i:73000
i:74000
i:75000
i:76000
i:77000
i:78000
i:79000
i:80000
i:81000
i:82000
i:83000
i:84000
i:85000
i:86000
i:87000
i:88000
i:89000
i:90000
i:91000
i:92000
i:93000
i:94000
i:95000
i:96000
i:97000
i:98000
i:99000
i:100000
i:101000
i:102000
i:103000
i:104000
i:105000
i:106000
i:107000
i:108000
i:109000
i:110000
i:111000
i:112000
i:113000
i:114000
i:115000
i:116000
i:117000
i:118000
i:119000
i:120000
i:121000
i:122000
i:123000
i:124000
i:125000
i:126000
i:127000
i:128000
i:129000
i:130000
i:131000
i:132000
i:133000
i:134000
i:135000
i:136000
i:1

In [23]:
np.max(Y)

16

In [24]:
len(np.unique(Y))

17

In [17]:
62 + 27 + 5 + 96 + 421 + 161 + 2347 + 354 + 1722 + 350 + 554 + 869 + 374 + 1535 + 145 + 2496

11518

In [16]:
np.unique(train_gt, return_counts=True)

(Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=uint8),
 array([11802882,       62,       27,        5,       96,      421,
             161,     2347,      354,     1722,      350,      554,
             869,      374,     1535,      145,     2496]))

In [ ]:
np.unique(test_gt, return_counts=True)

In [ ]:
np.unique(gt, return_counts=True)

In [ ]:
np.unique(Y, return_counts=True)

In [ ]:
_, pseudo_labels3 = sample_gt3(X, Y, train_gt, test_gt, SAMPLE_PERCENTAGE)